# Determining the Fair Spread of a CDS Tranche

An analysis of pricing a CDS Index using its intrinsic value

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.finutils import *
from financepy.products.rates import *
from financepy.products.credit import *

####################################################################
# FINANCEPY BETA Version 0.191 - This build:  17 Jan 2021 at 18:30 #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy  #
#      Send any bug reports or comments to quant@financepy.com     #
####################################################################



In [3]:
valuationDate = TuringDate(2, 8, 2007)
settlementDate = valuationDate.addWeekDays(1)

## Build Ibor Curve

In [4]:
dcType = TuringDayCountTypes.THIRTY_E_360_ISDA
depos = []
depo = TuringIborDeposit(valuationDate, "1D", 0.0500, dcType); depos.append(depo)
depo = TuringIborDeposit(settlementDate, "1D", 0.0500, dcType); depos.append(depo)

fixedFreq = TuringFrequencyTypes.SEMI_ANNUAL
swapType = TuringSwapTypes.PAY
swap1 = TuringIborSwap(settlementDate,"1Y",swapType,0.0502,fixedFreq,dcType)
swap2 = TuringIborSwap(settlementDate,"2Y",swapType,0.0502,fixedFreq,dcType)
swap3 = TuringIborSwap(settlementDate,"3Y",swapType,0.0501,fixedFreq,dcType)
swap4 = TuringIborSwap(settlementDate,"4Y",swapType,0.0502,fixedFreq,dcType)
swap5 = TuringIborSwap(settlementDate,"5Y",swapType,0.0501,fixedFreq,dcType)
swaps = [swap1,swap2,swap3,swap4,swap5]

liborCurve = TuringIborSingleCurve(valuationDate, depos, [], swaps)

We treat an index as a CDS contract with a flat CDS curve at the CDS index spread for the same maturity

## Create the Underlying CDS Index Portfolio

In [5]:
stepInDate = valuationDate.addWeekDays(1)

In [6]:
maturity3Y = valuationDate.nextCDSDate(36)
maturity5Y = valuationDate.nextCDSDate(60)
maturity7Y = valuationDate.nextCDSDate(84)
maturity10Y = valuationDate.nextCDSDate(120)

### Heterogeneous Curves

In [7]:
f = open('.//data//CDX_NA_IG_S7_SPREADS.csv', 'r')
data = f.readlines()
heteroIssuerCurves = []

numCredits = len(data) - 1  # The file has a header

for row in data[1:]:
    splitRow = row.split(",")
    spd3Y = float(splitRow[1]) / 10000.0
    spd5Y = float(splitRow[2]) / 10000.0
    spd7Y = float(splitRow[3]) / 10000.0
    spd10Y = float(splitRow[4]) / 10000.0
    recoveryRate = float(splitRow[5])
    cds3Y = TuringCDS(stepInDate, maturity3Y, spd3Y)
    cds5Y = TuringCDS(stepInDate, maturity5Y, spd5Y)
    cds7Y = TuringCDS(stepInDate, maturity7Y, spd7Y)
    cds10Y = TuringCDS(stepInDate, maturity10Y, spd10Y)
    cdsContracts = [cds3Y, cds5Y, cds7Y, cds10Y]
    issuerCurve = TuringCDSCurve(valuationDate, cdsContracts, liborCurve, recoveryRate)
    heteroIssuerCurves.append(issuerCurve)

### Homogeneous Curves 

Calculate the average spread of the heterogeneous portfolio

In [8]:
homoIssuerCurves = []
numCredits = 125
recoveryRate = 0.40 

In [9]:
cdsIndex = TuringCDSIndexPortfolio()

In [10]:
spd3Y = cdsIndex.intrinsicSpread(valuationDate, stepInDate, maturity3Y, heteroIssuerCurves)
spd5Y = cdsIndex.intrinsicSpread(valuationDate, stepInDate, maturity5Y, heteroIssuerCurves)
spd7Y = cdsIndex.intrinsicSpread(valuationDate, stepInDate, maturity7Y, heteroIssuerCurves)
spd10Y = cdsIndex.intrinsicSpread(valuationDate, stepInDate, maturity10Y, heteroIssuerCurves)

In [11]:
print("Homogeneous curve 3Y:", spd3Y*10000)
print("Homogeneous curve 5Y:", spd5Y*10000)
print("Homogeneous curve 7Y:", spd7Y*10000)
print("Homogeneous curve 10Y:", spd10Y*10000)

Homogeneous curve 3Y: 19.678791186198726
Homogeneous curve 5Y: 35.53904916872566
Homogeneous curve 7Y: 49.01163746623519
Homogeneous curve 10Y: 61.4133894203296


In [12]:
for row in range(0,numCredits):
    cds3Y = TuringCDS(stepInDate, maturity3Y, spd3Y)
    cds5Y = TuringCDS(stepInDate, maturity5Y, spd5Y)
    cds7Y = TuringCDS(stepInDate, maturity7Y, spd7Y)
    cds10Y = TuringCDS(stepInDate, maturity10Y, spd10Y)
    cdsContracts = [cds3Y, cds5Y, cds7Y, cds10Y]
    issuerCurve = TuringCDSCurve(valuationDate, cdsContracts, liborCurve, recoveryRate)
    homoIssuerCurves.append(issuerCurve)

## Define the Tranches

In [13]:
trancheMaturity = maturity5Y
tranche1 = TuringCDSTranche(valuationDate, trancheMaturity, 0.00, 0.03)
tranche2 = TuringCDSTranche(valuationDate, trancheMaturity, 0.03, 0.06)
tranche3 = TuringCDSTranche(valuationDate, trancheMaturity, 0.06, 0.09)
tranche4 = TuringCDSTranche(valuationDate, trancheMaturity, 0.09, 0.12)
tranche5 = TuringCDSTranche(valuationDate, trancheMaturity, 0.12, 0.22)
tranche6 = TuringCDSTranche(valuationDate, trancheMaturity, 0.22, 0.60)
tranche7 = TuringCDSTranche(valuationDate, trancheMaturity, 0.00, 0.60)
tranche8 = TuringCDSTranche(valuationDate, trancheMaturity, 0.00, 1.00)

In [14]:
tranches = [tranche1, tranche2, tranche3, tranche4, tranche5, tranche6, tranche7, tranche8]

In [15]:
corr1 = 0.30
corr2 = 0.30
upfront = 0.0
spd = 0.0

## Homogeneous Portfolio Results

In [16]:
print("%50s %5s %9s %9s %12s"% ("Method", "NumPts", "K1", "K2", "SPD(BPS)"))
for tranche in tranches:
    for method in TuringLossDistributionBuilder:
        for numPoints in [50]:
            v = tranche.valueBC(valuationDate,homoIssuerCurves,upfront,spd,corr1,corr2,numPoints,method)
            print("%50s %5d %9.5f %9.5f %12.6f"% (method, numPoints, tranche._k1*100, tranche._k2*100, v[3] * 10000))
    print("=============================================================================================")

                                            Method NumPts        K1        K2     SPD(BPS)
              TuringLossDistributionBuilder.RECURSION    50   0.00000   3.00000   875.142344
      TuringLossDistributionBuilder.ADJUSTED_BINOMIAL    50   0.00000   3.00000   875.142344
               TuringLossDistributionBuilder.GAUSSIAN    50   0.00000   3.00000   904.131917
                    TuringLossDistributionBuilder.LHP    50   0.00000   3.00000   914.428151
              TuringLossDistributionBuilder.RECURSION    50   3.00000   6.00000   239.744106
      TuringLossDistributionBuilder.ADJUSTED_BINOMIAL    50   3.00000   6.00000   239.744106
               TuringLossDistributionBuilder.GAUSSIAN    50   3.00000   6.00000   215.897269
                    TuringLossDistributionBuilder.LHP    50   3.00000   6.00000   226.978741
              TuringLossDistributionBuilder.RECURSION    50   6.00000   9.00000   102.226143
      TuringLossDistributionBuilder.ADJUSTED_BINOMIAL    50   6.00000   

## Heterogeneous Portfolio Results

In [17]:
print("%50s %5s %9s %9s %12s"% ("Method", "NumPts", "K1", "K2", "SPD(BPS)"))

for tranche in tranches:
    for method in TuringLossDistributionBuilder:
        for numPoints in [50]:
            v = tranche.valueBC(valuationDate,heteroIssuerCurves,upfront,spd,corr1,corr2,numPoints,method)
            print("%50s %5d  %9.5f %9.5f %12.6f"% (method, numPoints, tranche._k1*100, tranche._k2*100, v[3] * 10000))
    print("=============================================================================================")

                                            Method NumPts        K1        K2     SPD(BPS)
              TuringLossDistributionBuilder.RECURSION    50    0.00000   3.00000   949.643014
      TuringLossDistributionBuilder.ADJUSTED_BINOMIAL    50    0.00000   3.00000   949.577467
               TuringLossDistributionBuilder.GAUSSIAN    50    0.00000   3.00000   981.024090
                    TuringLossDistributionBuilder.LHP    50    0.00000   3.00000   914.815317
              TuringLossDistributionBuilder.RECURSION    50    3.00000   6.00000   231.078872
      TuringLossDistributionBuilder.ADJUSTED_BINOMIAL    50    3.00000   6.00000   231.166682
               TuringLossDistributionBuilder.GAUSSIAN    50    3.00000   6.00000   202.418752
                    TuringLossDistributionBuilder.LHP    50    3.00000   6.00000   226.986598
              TuringLossDistributionBuilder.RECURSION    50    6.00000   9.00000    87.300805
      TuringLossDistributionBuilder.ADJUSTED_BINOMIAL    50    

Copyright (c) 2020 Dominic O'Kane